In [ ]:
# ================================================================================ #
# =========================== Goolge Colab File Upload =========================== #
# ================================================================================ #

from google.colab import drive
drive.mount('/content/drive')

from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/MyDrive/Colab Notebooks/뉴스 토픽 분류 경진대회/open2.zip" "open2.zip"
# data.zip을 현재 디렉터리에 압축해제
!unzip "open2.zip"

!cp "/content/drive/MyDrive/Colab Notebooks/뉴스 토픽 분류 경진대회/ko.zip" "ko.zip"
!unzip "ko.zip"
output.clear()

In [ ]:
# import library
import numpy as np
import scipy as sp
import tensorflow as tf
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset

import pandas as pd
import pickle
import os
import glob
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split


# text data & label data load
PATH = "./"
df_train = pd.read_csv(PATH + "train_data.csv")
df_test = pd.read_csv(PATH + "test_data.csv")
topic_dict = pd.read_csv(PATH + "topic_dict.csv")
kr_stopwords_list = pd.read_csv(PATH + "korean_stopwords.csv", header = None, names = ['word'])["word"].tolist()

print("훈련 데이터 크기: ", df_train.shape)
print("예측 데이터 크기: ", df_test.shape)

훈련 데이터 크기:  (45654, 3)
예측 데이터 크기:  (9131, 2)


In [ ]:
# 한국어 형태소 및 문장 분석 라이브러리
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install nltk
!pip install konlpy

from konlpy.tag import Kkma
from pykospacing import Spacing
import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
output.clear()

In [ ]:
# dacon sample function
import re

def clean_text_dacon(text):
    text_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", text)
    text_clean = re.sub(r'\d+','',text_clean)
    text_clean = re.sub(r"^\s+", '', text_clean) #remove space from start 
    text_clean = re.sub(r'\s+$', '', text_clean) #remove space from the end corpus.append(review) return corpus
    text_clean = re.sub(r'<[^>]+>','',text_clean) #remove Html tags 
    return text_clean

# text_clean process
df_train["title_clean"] = df_train["title"].map(lambda text : clean_text_dacon(text))
df_test["title_clean"] = df_test["title"].map(lambda text : clean_text_dacon(text))

display(df_train[0:10])
display(df_test[0:10])

,index,title,topic_idx,title_clean
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천 핀란드 항공기 결항 휴가철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘밸리 넘어서겠다 구글 조원 들여 전역 거점화
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,클린턴 측근 기업 특수관계 조명 공과 사 맞물려종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망
5,5,팔레스타인 가자지구서 16세 소년 이스라엘군 총격에 사망,4,팔레스타인 가자지구서 세 소년 이스라엘군 총격에 사망
6,6,인도 48년 만에 파키스탄 공습…테러 캠프 폭격종합2보,4,인도 년 만에 파키스탄 공습 테러 캠프 폭격종합 보
7,7,美대선 TV토론 음담패설 만회실패 트럼프…사과 대신 빌클린턴 공격해 역효과,4,대선 토론 음담패설 만회실패 트럼프 사과 대신 빌클린턴 공격해 역효과
8,8,푸틴 한반도 상황 진전 위한 방안 김정은 위원장과 논의,4,푸틴 한반도 상황 진전 위한 방안 김정은 위원장과 논의
9,9,특검 면죄부 받은 트럼프 스캔들 보도 언론 맹공…국민의 적,4,특검 면죄부 받은 트럼프 스캔들 보도 언론 맹공 국민의 적


,index,title,title_clean
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영,유튜브 내달 일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사,어버이날 맑다가 흐려져 남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다,내년부터 국가 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,회색인간 작가 김동식 양심고백 등 새 소설집 권 출간
5,45659,야외서 생방송 하세요…액션캠 전용 요금제 잇따라,야외서 생방송 하세요 액션캠 전용 요금제 잇따라
6,45660,월드컵 태극전사 16강 전초기지 레오강 입성종합,월드컵 태극전사 강 전초기지 레오강 입성종합
7,45661,미세먼지 속 출근길,미세먼지 속 출근길
8,45662,왓츠앱稅 230원에 성난 레바논 민심…총리사퇴로 이어져종합2보,왓츠앱 원에 성난 레바논 민심 총리사퇴로 이어져종합 보
9,45663,베트남 경제 고성장 지속…2분기 GDP 6.71% 성장,베트남 경제 고성장 지속 분기 성장


In [ ]:
# bert model with transformers module
!git lfs install
!git clone https://huggingface.co/klue/bert-base

!pip install transformers
!pip install sentencepiece
output.clear()

In [ ]:
# model and data loaded
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels = 7)
print(model.config)

# tokenizer function for preprocessing
def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = False, 
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True   # Construct attn. masks.
        
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id


# train and valid data split
train_idx, valid_idx = train_test_split(df_train.index, shuffle = True, test_size = 0.1, random_state = 42)
trainlen = len(train_idx)
validlen = len(valid_idx)

MAXLEN = 12

input_ids = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
attention_masks = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
token_type_ids = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
train_data_labels = []

del_index = []

# train data tokenize(input_id, attention_mask, token_type_id)
for i, (train_sent, train_label) in enumerate(tqdm(zip(df_train["title_clean"].iloc[train_idx], df_train["topic_idx"].iloc[train_idx]), total= trainlen)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAXLEN)
        if len(input_id) != 0:
            input_id = np.array(input_id)
            attention_mask = np.array(attention_mask)
            token_type_id = np.array(token_type_id)

            input_ids[i,:] = input_id
            attention_masks[i,:] = attention_mask
            token_type_ids[i,:] = token_type_id
            train_data_labels.append(train_label)
        else:
            del_index.append(i)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

# 누락된 요소 제거
input_ids = np.delete(input_ids, del_index, axis = 0)
attention_masks = np.delete(attention_masks, del_index, axis = 0)
token_type_ids = np.delete(token_type_ids, del_index, axis = 0)

train_input_ids = torch.from_numpy(input_ids)
train_attention_masks = torch.from_numpy(attention_masks)
train_token_type_ids = torch.from_numpy(token_type_ids)

train_data_labels = np.array(train_data_labels, dtype = np.int32)
train_data_labels = torch.from_numpy(train_data_labels)

print("train_input_ids: ", train_input_ids.size())
print("train_attention_masks: ", train_attention_masks.size())
print("train_token_type_ids: ", train_token_type_ids.size())
print("train_data_labels : ", train_data_labels.size())

train_inputs = (train_input_ids, train_attention_masks, train_token_type_ids)

print("# sents: {}, # labels: {}".format(len(train_input_ids), len(train_data_labels)))


# valid data
input_ids = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
attention_masks = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
token_type_ids = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
valid_data_labels = []

del_index = []

for i, (valid_sent, valid_label) in enumerate(tqdm(zip(df_train["title_clean"].iloc[valid_idx], df_train["topic_idx"].iloc[valid_idx]), total= validlen)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(valid_sent, MAXLEN)
        if len(input_id) != 0:
            input_id = np.array(input_id)
            attention_mask = np.array(attention_mask)
            token_type_id = np.array(token_type_id)

            input_ids[i,:] = input_id
            attention_masks[i,:] = attention_mask
            token_type_ids[i,:] = token_type_id
            valid_data_labels.append(valid_label)
        else:
            del_index.append(i)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

# 누락된 요소 제거
input_ids = np.delete(input_ids, del_index, axis = 0)
attention_masks = np.delete(attention_masks, del_index, axis = 0)
token_type_ids = np.delete(token_type_ids, del_index, axis = 0)

valid_input_ids = torch.from_numpy(input_ids)
valid_attention_masks = torch.from_numpy(attention_masks)
valid_token_type_ids = torch.from_numpy(token_type_ids)

valid_data_labels = np.array(valid_data_labels, dtype = np.int32)
valid_data_labels = torch.from_numpy(valid_data_labels)

print("valid_input_ids: ", valid_input_ids.size())
print("valid_attention_masks: ", valid_attention_masks.size())
print("valid_token_type_ids: ", valid_token_type_ids.size())
print("valid_data_labels : ", valid_data_labels.size())

valid_inputs = (valid_input_ids, valid_attention_masks, valid_token_type_ids)

print("# sents: {}, # labels: {}".format(len(valid_input_ids), len(valid_data_labels)))

class BERTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, token_type_ids, labels):
      
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.token_type_ids = token_type_ids
        self.labels = labels

    def __getitem__(self, i):
        return (self.input_ids[i], self.attention_masks[i], self.token_type_ids[i], self.labels[i])

    def __len__(self):
        return (len(self.labels))

# dataloader generate
batch_size = 64

data_train = BERTDataset(train_input_ids, train_attention_masks, train_token_type_ids, train_data_labels)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 4)

data_valid = BERTDataset(valid_input_ids, valid_attention_masks, valid_token_type_ids, valid_data_labels)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size = batch_size, num_workers = 4)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'class

RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.9.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_si

 13%|█▎        | 590/4566 [00:00<00:00, 5898.14it/s]

train_input_ids:  torch.Size([41088, 12])
train_attention_masks:  torch.Size([41088, 12])
train_token_type_ids:  torch.Size([41088, 12])
train_data_labels :  torch.Size([41088])
# sents: 41088, # labels: 41088


100%|██████████| 4566/4566 [00:00<00:00, 5814.81it/s]

valid_input_ids:  torch.Size([4566, 12])
valid_attention_masks:  torch.Size([4566, 12])
valid_token_type_ids:  torch.Size([4566, 12])
valid_data_labels :  torch.Size([4566])
# sents: 4566, # labels: 4566


In [ ]:
# training model and validate the acc / loss

#parameter
learning_rate = 1e-5
num_epochs = 6
warmup_ratio = 0.25
log_interval = 128

# torch gpu allocation
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

print("device: ", device)
print("device nums: ", torch.cuda.device_count())

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1.0, )

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

from tqdm import tqdm, tqdm_notebook

!nvidia-smi # gpu allocation check
print(torch.cuda.memory_summary(device = device, abbreviated=False)) # cuda memory allocation summary printing

import gc
gc.collect()

model.to(device)
# 모델 학습 시작
for e in tqdm(range(num_epochs)):
    train_acc = 0.0
    test_acc = 0.0

    model.train()
    for batch_id, (input_ids, attention_masks, token_type_ids, labels) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        token_type_ids = token_type_ids.to(device)

        outputs = model.forward(input_ids, attention_masks, token_type_ids).logits
        labels = labels.long().to(device)
        loss = loss_fn(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # gradient clipping

        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(outputs, labels)

        del input_ids, attention_masks, token_type_ids # delete the training data
        torch.cuda.empty_cache() # gpu cache memory delete
        gc.collect()

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


    valid_acc = 0.0
    valid_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch_id, (input_ids, attention_masks, token_type_ids, labels) in enumerate(tqdm_notebook(valid_dataloader)):
            
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)

            outputs = model.forward(input_ids, attention_masks, token_type_ids).logits
            labels = labels.long().to(device)

            loss = loss_fn(outputs, labels)
            valid_acc += calc_accuracy(outputs, labels)
            valid_loss += loss

    print("epoch {} valid acc {} valid loss {}".format(e+1, valid_acc / (batch_id + 1), valid_loss / (batch_id +1)))

device:  cuda
device nums:  1
Fri Jul 23 14:44:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    34W / 250W |   6257MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                          

  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epoch 1 batch id 1 loss 2.010509729385376 train acc 0.09375
epoch 1 batch id 129 loss 0.6092613935470581 train acc 0.5853924418604651
epoch 1 batch id 257 loss 0.6085700988769531 train acc 0.6971060311284046
epoch 1 batch id 385 loss 0.6323629021644592 train acc 0.7405032467532467
epoch 1 batch id 513 loss 0.5501466989517212 train acc 0.763249269005848
epoch 1 batch id 641 loss 0.4843108654022217 train acc 0.7783492589703588

epoch 1 train acc 0.7783781152647975


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


 17%|█▋        | 1/6 [05:54<29:33, 354.72s/it]

...

 83%|████████▎ | 5/6 [29:38<05:55, 355.70s/it]


epoch 5 valid acc 0.8512073863636364 valid loss 0.49037256836891174


epoch 6 batch id 1 loss 0.10650146752595901 train acc 0.953125
epoch 6 batch id 129 loss 0.3427004814147949 train acc 0.9372577519379846
epoch 6 batch id 257 loss 0.13640815019607544 train acc 0.9367704280155642
epoch 6 batch id 385 loss 0.1974380612373352 train acc 0.9366883116883117
epoch 6 batch id 513 loss 0.15391430258750916 train acc 0.9372563352826511
epoch 6 batch id 641 loss 0.09909000992774963 train acc 0.9382069032761311

epoch 6 train acc 0.9381814641744548


100%|██████████| 6/6 [35:35<00:00, 355.98s/it]


epoch 6 valid acc 0.8520754419191919 valid loss 0.5366138219833374


In [ ]:
# submission
# test data

testlen = len(df_test)

input_ids = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)
attention_masks = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)
token_type_ids = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)


for i, test_sent in enumerate(tqdm(df_test["title_clean"], total = len(df_test))):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAXLEN)

        input_id = np.array(input_id)
        attention_mask = np.array(attention_mask)
        token_type_id = np.array(token_type_id)

        input_ids[i,:] = input_id
        attention_masks[i,:] = attention_mask
        token_type_ids[i,:] = token_type_id


    except Exception as e:
        print(e)
        print(test_sent)
        pass


# torch.tensor로 전환
test_input_ids = torch.from_numpy(input_ids.reshape(-1,MAXLEN))
test_attention_masks = torch.from_numpy(attention_masks.reshape(-1,MAXLEN))
test_token_type_ids = torch.from_numpy(token_type_ids.reshape(-1,MAXLEN))

class TESTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, token_type_ids):
      
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.token_type_ids = token_type_ids

    def __getitem__(self, i):
        return (self.input_ids[i], self.attention_masks[i], self.token_type_ids[i])

    def __len__(self):
        return (len(self.input_ids))

# dataset
data_test = TESTDataset(test_input_ids, test_attention_masks, test_token_type_ids)

# dataloader
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = 1, num_workers = 3)

predictions = []
model.eval()

with torch.no_grad():
    for batch_id, (input_ids, attention_masks, token_type_ids) in enumerate(test_dataloader):

        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        token_type_ids = token_type_ids.to(device)
        outputs = model.forward(input_ids, attention_masks, token_type_ids).logits
        predictions.append(torch.argmax(outputs, dim = 1).item())

    submission_topic = np.array(predictions)

from google.colab import files

def softmax_toIdx(predictions):
    topic_Idx = np.argmax(predictions, axis = 1).reshape(-1,1)
    return topic_Idx

# prediction
submission_topic = submission_topic.reshape(-1,1)

# submission 
submission = pd.read_csv(PATH + "sample_submission.csv")
submission["topic_idx"] = submission_topic
submission.to_csv("klue_roberta_submission.csv", index = False)
files.download("klue_roberta_submission.csv")

  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 9131/9131 [00:01<00:00, 6720.01it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# cross validation for kfold algorithm #

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels = 7)

# tokenizer function for preprocessing
def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = False, 
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True   # Construct attn. masks.
        
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

# datasets definition
class BERTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, token_type_ids, labels):
      
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.token_type_ids = token_type_ids
        self.labels = labels

    def __getitem__(self, i):
        return (self.input_ids[i], self.attention_masks[i], self.token_type_ids[i], self.labels[i])

    def __len__(self):
        return (len(self.labels))

from sklearn.model_selection import StratifiedKFold
n_folds = 3
kfold = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = 42)
MAXLEN = 16

# cross validation for train datasets

for cv, (train_cv, valid_cv) in enumerate(kfold.split(df_train["title_clean"], df_train["topic_idx"])):
    print("/// cross validation for cv : ", cv + 1)

    trainlen = len(train_cv)
    input_ids = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
    attention_masks = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
    token_type_ids = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
    train_data_labels = []
    del_index = []

    # train data tokenize(input_id, attention_mask, token_type_id)
    for i, (train_sent, train_label) in enumerate(tqdm(zip(df_train["title_clean"].iloc[train_cv], df_train["topic_idx"].iloc[train_cv]), total= trainlen)):
        try:
            input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAXLEN)
            if len(input_id) != 0:
                input_id = np.array(input_id)
                attention_mask = np.array(attention_mask)
                token_type_id = np.array(token_type_id)

                input_ids[i,:] = input_id
                attention_masks[i,:] = attention_mask
                token_type_ids[i,:] = token_type_id
                train_data_labels.append(train_label)
            else:
                del_index.append(i)

        except Exception as e:
            print(e)
            print(train_sent)
            pass


    # 누락된 요소 제거
    input_ids = np.delete(input_ids, del_index, axis = 0)
    attention_masks = np.delete(attention_masks, del_index, axis = 0)
    token_type_ids = np.delete(token_type_ids, del_index, axis = 0)

    train_input_ids = torch.from_numpy(input_ids)
    train_attention_masks = torch.from_numpy(attention_masks)
    train_token_type_ids = torch.from_numpy(token_type_ids)

    train_data_labels = np.array(train_data_labels, dtype = np.int32)
    train_data_labels = torch.from_numpy(train_data_labels)

    print("train_input_ids: ", train_input_ids.size())
    print("train_attention_masks: ", train_attention_masks.size())
    print("train_token_type_ids: ", train_token_type_ids.size())
    print("train_data_labels : ", train_data_labels.size())

    train_inputs = (train_input_ids, train_attention_masks, train_token_type_ids)

    print("# sents: {}, # labels: {}".format(len(train_input_ids), len(train_data_labels)))

    # valid data  먼저 확보하기
    validlen = len(valid_cv)
    input_ids = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
    attention_masks = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
    token_type_ids = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
    valid_data_labels = []

    del_index = []

    for i, (valid_sent, valid_label) in enumerate(tqdm(zip(df_train["title_clean"].iloc[valid_cv], df_train["topic_idx"].iloc[valid_cv]), total= len(valid_cv))):
        try:
            input_id, attention_mask, token_type_id = bert_tokenizer(valid_sent, MAXLEN)
            if len(input_id) != 0:
                input_id = np.array(input_id)
                attention_mask = np.array(attention_mask)
                token_type_id = np.array(token_type_id)

                input_ids[i,:] = input_id
                attention_masks[i,:] = attention_mask
                token_type_ids[i,:] = token_type_id
                valid_data_labels.append(valid_label)
            else:
                del_index.append(i)

        except Exception as e:
            print(e)
            print(train_sent)
            pass

    # 누락된 요소 제거
    input_ids = np.delete(input_ids, del_index, axis = 0)
    attention_masks = np.delete(attention_masks, del_index, axis = 0)
    token_type_ids = np.delete(token_type_ids, del_index, axis = 0)

    valid_input_ids = torch.from_numpy(input_ids)
    valid_attention_masks = torch.from_numpy(attention_masks)
    valid_token_type_ids = torch.from_numpy(token_type_ids)

    valid_data_labels = np.array(valid_data_labels, dtype = np.int32)
    valid_data_labels = torch.from_numpy(valid_data_labels)

    print("valid_input_ids: ", valid_input_ids.size())
    print("valid_attention_masks: ", valid_attention_masks.size())
    print("valid_token_type_ids: ", valid_token_type_ids.size())
    print("valid_data_labels : ", valid_data_labels.size())

    valid_inputs = (valid_input_ids, valid_attention_masks, valid_token_type_ids)

    print("valid /// # sents: {}, # labels: {}".format(len(valid_input_ids), len(valid_data_labels)))

    data_valid = BERTDataset(valid_input_ids, valid_attention_masks, valid_token_type_ids, valid_data_labels)
    valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size = batch_size, num_workers = 4)


    # dataloader generate
    batch_size = 64

    data_train = BERTDataset(train_input_ids, train_attention_masks, train_token_type_ids, train_data_labels)
    train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 4)

    # training model and validate the acc / loss

    #parameter
    learning_rate = 3e-5
    num_epochs = 5
    warmup_ratio = 0.2
    log_interval = 200

    # torch gpu allocation
    USE_CUDA = torch.cuda.is_available()
    device = torch.device("cuda" if USE_CUDA else "cpu")

    print("device: ", device)
    print("device nums: ", torch.cuda.device_count())

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    # 옵티마이저 선언
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

    t_total = len(train_dataloader) * num_epochs
    warmup_step = int(t_total * warmup_ratio)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1.0, )

    # 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
    def calc_accuracy(X,Y):
        max_vals, max_indices = torch.max(X, 1)
        train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
        return train_acc

    from tqdm import tqdm, tqdm_notebook
    import gc
    gc.collect()

    model.to(device)
    # 모델 학습 시작
    for e in tqdm(range(num_epochs)):
        train_acc = 0.0
        test_acc = 0.0

        model.train()
        for batch_id, (input_ids, attention_masks, token_type_ids, labels) in enumerate(tqdm_notebook(train_dataloader)):
            optimizer.zero_grad()
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)

            outputs = model.forward(input_ids, attention_masks, token_type_ids).logits
            labels = labels.long().to(device)
            loss = loss_fn(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # gradient clipping

            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            train_acc += calc_accuracy(outputs, labels)

            del input_ids, attention_masks, token_type_ids # delete the training data
            torch.cuda.empty_cache() # gpu cache memory delete
            gc.collect()

            if batch_id % log_interval == 0:
                print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


        valid_acc = 0.0
        valid_loss = 0.0
        model.eval()
        with torch.no_grad():
            for batch_id, (input_ids, attention_masks, token_type_ids, labels) in enumerate(tqdm_notebook(valid_dataloader)):
                
                input_ids = input_ids.to(device)
                attention_masks = attention_masks.to(device)
                token_type_ids = token_type_ids.to(device)

                outputs = model.forward(input_ids, attention_masks, token_type_ids).logits
                labels = labels.long().to(device)

                loss = loss_fn(outputs, labels)
                valid_acc += calc_accuracy(outputs, labels)
                valid_loss += loss

                if batch_id % log_interval == 0:
                    print("epoch {} batch id {} loss {} valid acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), valid_acc / (batch_id+1)))

            print("epoch {} valid acc {} valid loss {}".format(e+1, valid_acc / (batch_id + 1), valid_loss / (batch_id +1)))

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'class

/// cross validation for cv :  1






  4%|▍         | 1369/30436 [00:00<00:04, 6850.74it/s]



  7%|▋         | 2046/30436 [00:00<00:04, 6826.19it/s]



  9%|▉         | 2730/30436 [00:00<00:04, 6829.19it/s]



 11%|█         | 3402/30436 [00:00<00:03, 6793.29it/s]



 13%|█▎        | 4090/30436 [00:00<00:03, 6816.72it/s]



 16%|█▌        | 4759/30436 [00:00<00:03, 6775.84it/s]



 18%|█▊        | 5438/30436 [00:00<00:03, 6777.96it/s]



 20%|██        | 6116/30436 [00:00<00:03, 6778.13it/s]



 22%|██▏       | 6767/30436 [00:01<00:03, 6646.62it/s]



 24%|██▍       | 7426/30436 [00:01<00:03, 6628.99it/s]



 27%|██▋       | 8120/30436 [00:01<00:03, 6717.51it/s]



 29%|██▉       | 8802/30436 [00:01<00:03, 6745.77it/s]



 31%|███       | 9496/30436 [00:01<00:03, 6800.41it/s]



 33%|███▎      | 10181/30436 [00:01<00:02, 6813.85it/s]



 36%|███▌      | 10863/30436 [00:01<00:02, 6815.03it/s]



 38%|███▊      | 11558/30436 [00:01<00:02, 6854.10it/s]



 40%|████      | 12243/30436 [00:01<00:02, 6821.72it/s]



 42%|█

train_input_ids:  torch.Size([30436, 16])
train_attention_masks:  torch.Size([30436, 16])
train_token_type_ids:  torch.Size([30436, 16])
train_data_labels :  torch.Size([30436])
# sents: 30436, # labels: 30436






  9%|▉         | 1344/15218 [00:00<00:02, 6820.94it/s]



 13%|█▎        | 2036/15218 [00:00<00:01, 6849.36it/s]



 18%|█▊        | 2706/15218 [00:00<00:01, 6801.69it/s]



 22%|██▏       | 3387/15218 [00:00<00:01, 6802.18it/s]



 26%|██▋       | 4028/15218 [00:00<00:01, 6679.28it/s]



 31%|███       | 4708/15218 [00:00<00:01, 6711.09it/s]



 35%|███▌      | 5395/15218 [00:00<00:01, 6756.55it/s]



 40%|████      | 6096/15218 [00:00<00:01, 6828.91it/s]



 44%|████▍     | 6752/15218 [00:01<00:01, 6729.56it/s]



 49%|████▉     | 7441/15218 [00:01<00:01, 6776.81it/s]



 53%|█████▎    | 8129/15218 [00:01<00:01, 6805.69it/s]



 58%|█████▊    | 8811/15218 [00:01<00:00, 6809.50it/s]



 62%|██████▏   | 9486/15218 [00:01<00:00, 6696.65it/s]



 67%|██████▋   | 10152/15218 [00:01<00:00, 6676.16it/s]



 71%|███████   | 10830/15218 [00:01<00:00, 6704.70it/s]



 76%|███████▌  | 11499/15218 [00:01<00:00, 6639.25it/s]



 80%|███████▉  | 12174/15218 [00:01<00:00, 6671.29it/s]



 84%|█

...


epoch 4 valid acc 0.9835871848739496 valid loss 0.07234302908182144


epoch 5 batch id 1 loss 0.23398374021053314 train acc 0.9375
epoch 5 batch id 201 loss 0.0030196465086191893 train acc 0.9803327114427861
epoch 5 batch id 401 loss 0.04073327034711838 train acc 0.9840632793017456

epoch 5 train acc 0.9846047794117647


epoch 5 batch id 1 loss 0.11283619701862335 valid acc 0.984375
epoch 5 batch id 201 loss 0.029913535341620445 valid acc 0.9818097014925373






100%|██████████| 5/5 [28:00<00:00, 336.11s/it]


epoch 5 valid acc 0.9827809873949579 valid loss 0.07608602195978165


30436

In [ ]:
# submission
# test data

testlen = len(df_test)

input_ids = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)
attention_masks = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)
token_type_ids = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)


for i, test_sent in enumerate(tqdm(df_test["title_clean"], total = len(df_test))):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAXLEN)

        input_id = np.array(input_id)
        attention_mask = np.array(attention_mask)
        token_type_id = np.array(token_type_id)

        input_ids[i,:] = input_id
        attention_masks[i,:] = attention_mask
        token_type_ids[i,:] = token_type_id


    except Exception as e:
        print(e)
        print(test_sent)
        pass


# torch.tensor로 전환
test_input_ids = torch.from_numpy(input_ids.reshape(-1,MAXLEN))
test_attention_masks = torch.from_numpy(attention_masks.reshape(-1,MAXLEN))
test_token_type_ids = torch.from_numpy(token_type_ids.reshape(-1,MAXLEN))

class TESTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, token_type_ids):
      
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.token_type_ids = token_type_ids

    def __getitem__(self, i):
        return (self.input_ids[i], self.attention_masks[i], self.token_type_ids[i])

    def __len__(self):
        return (len(self.input_ids))

# dataset
data_test = TESTDataset(test_input_ids, test_attention_masks, test_token_type_ids)

# dataloader
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = 1, num_workers = 3)

predictions = []
model.eval()

with torch.no_grad():
    for batch_id, (input_ids, attention_masks, token_type_ids) in enumerate(test_dataloader):

        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        token_type_ids = token_type_ids.to(device)
        outputs = model.forward(input_ids, attention_masks, token_type_ids).logits
        predictions.append(torch.argmax(outputs, dim = 1).item())

    submission_topic = np.array(predictions)

from google.colab import files

def softmax_toIdx(predictions):
    topic_Idx = np.argmax(predictions, axis = 1).reshape(-1,1)
    return topic_Idx

# prediction
submission_topic = submission_topic.reshape(-1,1)

# submission 
submission = pd.read_csv(PATH + "sample_submission.csv")
submission["topic_idx"] = submission_topic
submission.to_csv("klue_roberta_cv_submission.csv", index = False)
files.download("klue_roberta_cv_submission.csv")





  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,




  7%|▋         | 682/9131 [00:00<00:01, 6816.71it/s]



 15%|█▍        | 1358/9131 [00:00<00:01, 6799.13it/s]



 22%|██▏       | 2044/9131 [00:00<00:01, 6815.71it/s]



 30%|██▉       | 2705/9131 [00:00<00:00, 6750.72it/s]



 37%|███▋      | 3372/9131 [00:00<00:00, 6724.03it/s]



 44%|████▍     | 4026/9131 [00:00<00:00, 6664.53it/s]



 51%|█████     | 4674/9131 [00:00<00:00, 6606.22it/s]



 59%|█████▊    | 5351/9131 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# fine tuning with ensemble model(klue-roberta + gru model)
from transformers import AutoTokenizer, AutoModel
#header_model = AutoModelForMaskedLM.from_pretrained("klue/roberta-large")

torch.cuda.empty_cache()
header_model = AutoModel.from_pretrained("klue/roberta-large", output_attentions = False)
header_model.cuda()
#outputs = header_model.forward(input_ids, attention_masks, token_type_ids)
#print(outputs["last_hidden_state"])
#print(outputs.last_hidden_state.size())

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

# tokenizer function for preprocessing
def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = False, 
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True   # Construct attn. masks.
        
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [ ]:
# model analysis
print(header_model.children)

<bound method Module.children of RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 1024, padding_idx=1)
    (position_embeddings): Embedding(512, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elemen

In [ ]:
# train and valid data split
train_idx, valid_idx = train_test_split(df_train.index, shuffle = True, test_size = 0.1, random_state = 42)
trainlen = len(train_idx)
validlen = len(valid_idx)

MAXLEN = 32

input_ids = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
attention_masks = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
token_type_ids = np.zeros(shape = (trainlen, MAXLEN), dtype = np.int32)
train_data_labels = []

del_index = []

# train data tokenize(input_id, attention_mask, token_type_id)
for i, (train_sent, train_label) in enumerate(tqdm(zip(df_train["title_clean"].iloc[train_idx], df_train["topic_idx"].iloc[train_idx]), total= trainlen)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAXLEN)
        if len(input_id) != 0:
            input_id = np.array(input_id)
            attention_mask = np.array(attention_mask)
            token_type_id = np.array(token_type_id)

            input_ids[i,:] = input_id
            attention_masks[i,:] = attention_mask
            token_type_ids[i,:] = token_type_id
            train_data_labels.append(train_label)
        else:
            del_index.append(i)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

# 누락된 요소 제거
input_ids = np.delete(input_ids, del_index, axis = 0)
attention_masks = np.delete(attention_masks, del_index, axis = 0)
token_type_ids = np.delete(token_type_ids, del_index, axis = 0)

train_input_ids = torch.from_numpy(input_ids)
train_attention_masks = torch.from_numpy(attention_masks)
train_token_type_ids = torch.from_numpy(token_type_ids)

train_data_labels = np.array(train_data_labels, dtype = np.int32)
train_data_labels = torch.from_numpy(train_data_labels)

print("train_input_ids: ", train_input_ids.size())
print("train_attention_masks: ", train_attention_masks.size())
print("train_token_type_ids: ", train_token_type_ids.size())
print("train_data_labels : ", train_data_labels.size())

train_inputs = (train_input_ids, train_attention_masks, train_token_type_ids)

print("# sents: {}, # labels: {}".format(len(train_input_ids), len(train_data_labels)))


# valid data
input_ids = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
attention_masks = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
token_type_ids = np.zeros(shape = (validlen, MAXLEN), dtype = np.int32)
valid_data_labels = []

del_index = []

for i, (valid_sent, valid_label) in enumerate(tqdm(zip(df_train["title_clean"].iloc[valid_idx], df_train["topic_idx"].iloc[valid_idx]), total= validlen)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(valid_sent, MAXLEN)
        if len(input_id) != 0:
            input_id = np.array(input_id)
            attention_mask = np.array(attention_mask)
            token_type_id = np.array(token_type_id)

            input_ids[i,:] = input_id
            attention_masks[i,:] = attention_mask
            token_type_ids[i,:] = token_type_id
            valid_data_labels.append(valid_label)
        else:
            del_index.append(i)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

# 누락된 요소 제거
input_ids = np.delete(input_ids, del_index, axis = 0)
attention_masks = np.delete(attention_masks, del_index, axis = 0)
token_type_ids = np.delete(token_type_ids, del_index, axis = 0)

valid_input_ids = torch.from_numpy(input_ids)
valid_attention_masks = torch.from_numpy(attention_masks)
valid_token_type_ids = torch.from_numpy(token_type_ids)

valid_data_labels = np.array(valid_data_labels, dtype = np.int32)
valid_data_labels = torch.from_numpy(valid_data_labels)

print("valid_input_ids: ", valid_input_ids.size())
print("valid_attention_masks: ", valid_attention_masks.size())
print("valid_token_type_ids: ", valid_token_type_ids.size())
print("valid_data_labels : ", valid_data_labels.size())

valid_inputs = (valid_input_ids, valid_attention_masks, valid_token_type_ids)

print("# sents: {}, # labels: {}".format(len(valid_input_ids), len(valid_data_labels)))

class BERTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, token_type_ids, labels):
      
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.token_type_ids = token_type_ids
        self.labels = labels

    def __getitem__(self, i):
        return (self.input_ids[i], self.attention_masks[i], self.token_type_ids[i], self.labels[i])

    def __len__(self):
        return (len(self.labels))

# dataloader generate
batch_size = 64

data_train = BERTDataset(train_input_ids, train_attention_masks, train_token_type_ids, train_data_labels)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 4)

data_valid = BERTDataset(valid_input_ids, valid_attention_masks, valid_token_type_ids, valid_data_labels)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size = batch_size, num_workers = 4)

# (batch, maxlen, ff_dim) -> (batch, maxlen, ff_dim)
# 실제 모델에서는 (batch_size, MAXLEN, 1024) inputs

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim = 1024, num_heads = 16, ff_dim = 256, rate = 0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim, num_heads, dropout = rate, kdim = embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim,ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim),
            nn.ReLU()
        )
        self.layernorm1 = nn.LayerNorm([MAXLEN, 1024])
        self.layernorm2 = nn.LayerNorm([MAXLEN, 1024])
        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)
        
    def forward(self, inputs):
        att_output, _ = self.att(inputs, inputs, inputs)
        att_output = self.dropout1(att_output)
        out1 = self.layernorm1(inputs + att_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)


# using lstm model
class BertClassifier(nn.Module):
    def __init__(self, lstm_hidden_dim = 128, lstm_num_layers = 1, num_class = 7, bidirectional = False, fc_hidden_dim = 64, dropout = 0.2):
        super(BertClassifier, self).__init__()
        self.bertModel = header_model
        self.bert_drop = nn.Dropout(0.25)
        # lstm model
        self.lstm = nn.LSTM(1024, lstm_hidden_dim, lstm_num_layers, bidirectional = bidirectional, batch_first = True)
        self.lstm_norm = nn.LayerNorm([MAXLEN, lstm_hidden_dim])
        self.lstm_relu = nn.ReLU()
        self.lstm_drop = nn.Dropout(0.25)
        self.fc1 = nn.Linear(lstm_hidden_dim * 2 if bidirectional else lstm_hidden_dim, fc_hidden_dim)
        self.fc1_norm = nn.LayerNorm(fc_hidden_dim)
        self.fc1_relu = nn.ReLU()
        self.fc1_drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(fc_hidden_dim, num_class)
 
    def forward(self, input_ids, attention_masks, token_type_ids):
        lhs = self.bertModel.forward(input_ids, attention_masks, token_type_ids).last_hidden_state
        x = self.bert_drop(lhs)
        x, (h, c) = self.lstm(x)
        x = self.lstm_norm(x)
        x = self.lstm_relu(x)
        x = self.lstm_drop(x)
        x = self.fc1(x[:, -1, :])
        x = self.fc1_norm(x)
        x = self.fc1_drop(x)
        x = self.fc2(x)

        return x


'''
# using transformer block
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bertModel = header_model
        self.bert_drop = nn.Dropout(0.25)
        self.transformer_block = TransformerBlock()
        self.pooling = nn.AdaptiveAvgPool1d(32)
        self.pooling_drop = nn.Dropout(0.25)
        self.fc1 = nn.Linear(MAXLEN * 32, 64)
        self.fc1_drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(64, 7)

    def forward(self, input_ids, attention_masks, token_type_ids):
        x = self.bertModel.forward(input_ids, attention_masks, token_type_ids).last_hidden_state
        x = self.bert_drop(x)
        x = self.transformer_block(x)
        x = self.pooling(x)
        x = self.pooling_drop(x)
        x = x.view(x.size(0), -1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc1_drop(x)
        outputs = self.fc2(x)

        return outputs
'''

bert_classifier = BertClassifier(128,1,7,False, 64, 0.2)
#bert_classifier = BertClassifier()

# training model and validate the acc / loss

#parameter
learning_rate = 1e-4
num_epochs = 8
warmup_ratio = 0.2
log_interval = 16

# torch gpu allocation
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

print("device: ", device)
print("device nums: ", torch.cuda.device_count())

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in bert_classifier.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in bert_classifier.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1.0, )

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

from tqdm import tqdm, tqdm_notebook

!nvidia-smi # gpu allocation check
print(torch.cuda.memory_summary(device = device, abbreviated=False)) # cuda memory allocation summary printing

import gc
gc.collect()

bert_classifier.to(device)

# 모델 학습 시작
for e in tqdm(range(num_epochs)):
    train_acc = 0.0
    test_acc = 0.0

    bert_classifier.train()
    for batch_id, (input_ids, attention_masks, token_type_ids, labels) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        token_type_ids = token_type_ids.to(device)

        outputs = bert_classifier.forward(input_ids, attention_masks, token_type_ids)
        labels = labels.long().to(device)
        loss = loss_fn(outputs, labels)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # gradient clipping

        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(outputs, labels)

        del input_ids, attention_masks, token_type_ids # delete the training data
        torch.cuda.empty_cache() # gpu cache memory delete
        gc.collect()

        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


    valid_acc = 0.0
    valid_loss = 0.0
    bert_classifier.eval()
    with torch.no_grad():
        for batch_id, (input_ids, attention_masks, token_type_ids, labels) in enumerate(tqdm_notebook(valid_dataloader)):
            
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            token_type_ids = token_type_ids.to(device)

            outputs = bert_classifier.forward(input_ids, attention_masks, token_type_ids)
            labels = labels.long().to(device)

            loss = loss_fn(outputs, labels)
            valid_acc += calc_accuracy(outputs, labels)
            valid_loss += loss

    print("epoch {} valid acc {} valid loss {}".format(e+1, valid_acc / (batch_id + 1), valid_loss / (batch_id +1)))

In [ ]:
# using transformer block
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bertModel = header_model
        self.bert_drop = nn.Dropout(0.25)
        self.transformer_block = TransformerBlock()
        self.pooling = nn.AdaptiveAvgPool1d(32)
        self.pooling_drop = nn.Dropout(0.25)
        self.fc1 = nn.Linear(MAXLEN, 32)
        self.fc1_drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(32, 7)

    def forward(self, input_ids, attention_masks, token_type_ids):
        x = self.bertModel.forward(input_ids, attention_masks, token_type_ids).last_hidden_state
        x = self.bert_drop(x)
        x = self.transformer_block(x)
        print(x.size())
        x = self.pooling(x)
        print(x.size())
        x = self.pooling_drop(x)
        print(x.size())
        x = x.view(x.size(0), -1)
        print(x.size())
        x = torch.nn.functional.relu(self.fc1(x))
        print(x.size())
        x = self.fc1_drop(x)
        print(x.size())
        outputs = self.fc2(x)

        return outputs

temp = BertClassifier().cuda()
output_temp = temp(input_ids, attention_masks, token_type_ids)

torch.Size([32, 16, 1024])
torch.Size([32, 16, 32])
torch.Size([32, 16, 32])
torch.Size([32, 512])
torch.Size([32, 32])
torch.Size([32, 32])


In [ ]:
# submission
# test data

testlen = len(df_test)

input_ids = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)
attention_masks = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)
token_type_ids = np.zeros(shape = (testlen, MAXLEN), dtype = np.int32)

for i, test_sent in enumerate(tqdm(df_test["title_clean"], total = len(df_test))):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAXLEN)

        input_id = np.array(input_id)
        attention_mask = np.array(attention_mask)
        token_type_id = np.array(token_type_id)

        input_ids[i,:] = input_id
        attention_masks[i,:] = attention_mask
        token_type_ids[i,:] = token_type_id


    except Exception as e:
        print(e)
        print(test_sent)
        pass


# torch.tensor로 전환
test_input_ids = torch.from_numpy(input_ids.reshape(-1,MAXLEN))
test_attention_masks = torch.from_numpy(attention_masks.reshape(-1,MAXLEN))
test_token_type_ids = torch.from_numpy(token_type_ids.reshape(-1,MAXLEN))

class TESTDataset(Dataset):
    def __init__(self, input_ids, attention_masks, token_type_ids):
      
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.token_type_ids = token_type_ids

    def __getitem__(self, i):
        return (self.input_ids[i], self.attention_masks[i], self.token_type_ids[i])

    def __len__(self):
        return (len(self.input_ids))

# dataset
data_test = TESTDataset(test_input_ids, test_attention_masks, test_token_type_ids)

# dataloader
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = 1, num_workers = 3)

predictions = []
bert_classifier.eval()
with torch.no_grad():
    for batch_id, (input_ids, attention_masks, token_type_ids) in enumerate(test_dataloader):

        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        token_type_ids = token_type_ids.to(device)
        outputs = bert_classifier.forward(input_ids, attention_masks, token_type_ids)
        predictions.append(torch.argmax(outputs, dim = 1).item())

    submission_topic = np.array(predictions)

from google.colab import files

def softmax_toIdx(predictions):
    topic_Idx = np.argmax(predictions, axis = 1).reshape(-1,1)
    return topic_Idx

# prediction
submission_topic = submission_topic.reshape(-1,1)

# submission 
submission = pd.read_csv(PATH + "sample_submission.csv")
submission["topic_idx"] = submission_topic
submission.to_csv("klue_roberta_lstm_submission.csv", index = False)
files.download("klue_roberta_lstm_submission.csv")